# 第六章 连接

In [1]:
import numpy as np
import pandas as pd

## 一、关系型连接
### 1.连接的基本概念
把两张相关的表按照某一个或某一组键连接起来是一种常见操作，在关系型连接中，键 是十分重要的，往往用 on 参数表示

在 pandas 中的关系型连接函数 merge 和 join 中提供了 how 参数来代表连接形式，分为左连接 left 、右连接 right 、内连接 inner 、外连接 outer 。左连接即以左边的键为准，如果右边表中的键于左边存在，那么就添加到左边，否则则处理为缺失值，右连接类似处理。内连接只负责合并两边同时出现的键，而外连接则会在内连接的基础上包含只在左边出现以及只在右边出现的值，因此外连接又叫全连接。

设左表中键 张三 出现两次，右表中的 张三 也出现两次，那么逐个进行匹配，最后产生的表必然包含 2*2 个姓名为 张三 的行。

### 2.值连接
- merge函数

In [3]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'],'Gender':['F','M']})
df1.merge(df2,on='Name',how='left')#这种连接还是要以df1的name为主

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


如果两个表中想要连接的列不具备相同的列名，可以通过 left_on 和 right_on 指定

In [4]:
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'],'Gender':['F','M']})
df1.merge(df2,left_on='df1_name',right_on='df2_name',how='left')

,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


如果两个表中的列出现了重复的列名，那么可以通过 suffixes 参数指定

In [5]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


某些时候出现重复元素是麻烦的，例如两位同学来自不同的班级，但是姓名相同，这种时候就要指定 on 参数为多个列使得正确连接

In [7]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Age':[20, 21],'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Gender':['F', 'M'],'Class':['two', 'one']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [8]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,one


In [9]:
df1.merge(df2, on='Name', how='left') # 错误的结果

,Name,Age,Class_x,Gender,Class_y
0,San Zhang,20,one,F,two
1,San Zhang,20,one,M,one
2,San Zhang,21,two,F,two
3,San Zhang,21,two,M,one


In [10]:
df1.merge(df2, on=['Name', 'Class'], how='left') # 正确的结果

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


键得是唯一的，为了保证唯一性。除了用duplicted检查是否重复外， merge 中也提供了 validate 参数来检查连接的唯一性模式。这里共有三种模式，即一对一连接 1:1 ，一对多连接 1:m ，多对一连接 m:1 连接，第一个是指左右表的键都是唯一的，后面两个分别指左表键唯一和右表键唯一

#### 【练一练】
上面以多列为键的例子中，错误写法显然是一种多对多连接，而正确写法是一对一连接，请修改原表，使得以多列为键的正确写法能够通过 validate='1:m' 的检验，但不能通过 validate='m:1' 的检验。

In [15]:
df1.merge(df2, on=['Name', 'Class'], how='left',validate='1:1')#df1与df2里面基于on的组合都是唯一的

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


In [17]:
df2=pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Gender':['F', 'M'],'Class':['two', 'two']})
df2#修改df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,two


In [21]:
df1.merge(df2, on=['Name', 'Class'], how='left',validate='1:m')#1:m，df1的键是唯一的，但df2不是

,Name,Age,Class,Gender
0,San Zhang,20,one,NaN
1,San Zhang,21,two,F
2,San Zhang,21,two,M


In [23]:
#df1.merge(df2, on=['Name', 'Class'], how='left',validate='m:1')这样写就会报错

### 3.索引连接
索引连接，就是把索引当作键，因此这和值连接本质上没有区别， pandas 中利用 join 函数来处理索引连接，它的参数选择要少于 merge ，除了必须的 on 和 how 之外，可以对重复的列指定左右后缀 lsuffix 和 rsuffix 。其中， on 参数指索引名

- join

In [25]:
df1 = pd.DataFrame({'Age':[20,30]},index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']},index=pd.Series(['Si Li','Wu Wang'],name='Name'))
df1.join(df2, how='left')

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


In [26]:
df1 = pd.DataFrame({'Grade':[70]},index=pd.Series(['San Zhang'],name='Name'))
df2 = pd.DataFrame({'Grade':[80]},index=pd.Series(['San Zhang'],name='Name'))
df1.join(df2, how='left', lsuffix='_Chinese', rsuffix='_Math')

,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


如果想要进行类似于 merge 中以多列为键的操作的时候， join 需要使用多级索引

In [28]:
df1 = pd.DataFrame({'Age':[20,21]},index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['one', 'two']],names=('Name','Class')))
df2 = pd.DataFrame({'Gender':['F', 'M']},index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['two', 'one']],names=('Name','Class')))
df1

Age
Name      Class     
San Zhang one     20
          two     21

In [29]:
df2

Gender
Name      Class       
San Zhang two        F
          one        M

In [30]:
df1.join(df2)

Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

## 二、方向连接
### 1.concat

有时候用户并不关心以哪一列为键来合并，只是希望把两个表或者多个表按照纵向或者横向拼接，为这种需求， pandas 中提供了 concat 函数来实现。
在 concat 中，最常用的有三个参数，它们是 axis, join, keys ，分别表示拼接方向，连接形式，以及在新表中指示来自于哪一张旧表的名字。这里需要特别注意， join 和 keys 与之前提到的 join 函数和键的概念没有任何关系。
 axis=0 ，表示纵向拼接多个表，常常用于多个样本的拼接；而 axis=1 表示横向拼接多个表，常用于多个字段或特征的拼接。

In [32]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1,df2])#纵向拼接

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [34]:
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1,df2,df3],1)#横向拼接

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


In [35]:
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})
pd.concat([df1, df2])#默认状态下join为outer，表示保留所有的列，并将不存在的值设为缺失

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
0,Wu Wang,NaN,M


In [36]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2])
pd.concat([df1, df2], 1)#这种设置行索引要根据索引对齐

,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [38]:
pd.concat([df1, df2], axis=1, join='inner')#join=inner ，表示保留两个表都出现过的列。横向拼接则根据行索引对齐

,Name,Age,Grade
1,Si Li,30,80


 keys 参数的使用场景在于多个表合并后，用户仍然想要知道新表中的数据来自于哪个原表，这时可以通过 keys 参数产生多级索引进行标记

In [40]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
pd.concat([df1, df2], keys=['one', 'two'])#通过keys生成了多级索引

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

### 2.序列与表的合并
- append：把一个序列追加到表的行末

在 append 中，如果原表是默认整数序列的索引，那么可以使用 ignore_index=True 对新序列对应索引的自动标号，否则必须对 Series 指定 name 属性。

In [43]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s, ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


- assign：把一个序列追加到表的列末

In [44]:
s = pd.Series([80, 90])
df1.assign(Grade=s)#assign返回一个临时副本

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


In [46]:
df1['Grade'] = s
df1#用【】修改是直接在原表上进行改动

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


## 三、类连接操作
### 1.比较

In [48]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],'Age':[20, 21 ,21],'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],'Age':[20, 21 ,21],'Class':['one', 'two', 'Three']})
df1.compare(df2)
#结果中返回了不同值所在的行列，如果相同则会被填充为缺失值 NaN ，其中 other 和 self 分别指代传入的参数表和被调用的表自身。

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

In [49]:
df1.compare(df2, keep_shape=True)#比较所有的元素的

Name         Age        Class       
    self  other self other   self  other
0    NaN    NaN  NaN   NaN    NaN    NaN
1  Si Li  Li Si  NaN   NaN    NaN    NaN
2    NaN    NaN  NaN   NaN  three  Three

### 2.组合
- combine

In [57]:
def choose_min(s1, s2):#选出较小的值，nan意味着0，最小
    s2=s2.reindex_like(s1)
    res=s1.where(s1<s2,s2)
    res=res.mask(s1.isna())
    return res

df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)#依次传入ABCD四列，s1是df1的某列，s2是df2的某列

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 【练一练】
请在上述代码的基础上修改，保留 df2 中4个未被 df1 替换的相应位置原始值。

In [58]:
def choose_min(s1, s2):#选出较小的值，nan意味着0，最小
    s2=s2.reindex_like(s1)
    res=s1.where(s1<s2,s2)
    #res=res.mask(s1.isna())
    return res

df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)#依次传入ABCD四列，s1是df1的某列，s2是df2的某列

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,9.0
2,NaN,6.0,8.0,10.0


#### 【end】

In [60]:
def choose_min(s1, s2):#选出较小的值，nan意味着0，最小
    s2=s2.reindex_like(s1)
    res=s1.where(s1<s2,s2)
    res=res.mask(s1.isna())
    return res

In [61]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


### 【练一练】
除了 combine 之外， pandas 中还有一个 combine_first 方法，其功能是在对两张表组合时，若第二张表中的值在第一张表中对应索引位置的值不是缺失状态，那么就使用第一张表的值填充。下面给出一个例子，请用 combine 函数完成相同的功能。

In [62]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


In [67]:
def choose_my(s1, s2):#当s1为nan时，将s2的元素取来替代nan
    res=s1.mask(np.isnan(s1),s2)
    return res

df1.combine(df2, choose_my)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


## 四、练习
### EX1：美国疫情数据集
现有美国4月12日至11月16日的疫情报表，请将 New York 的 Confirmed, Deaths, Recovered, Active 合并为一张表，索引为按如下方法生成的日期字符串序列：

In [72]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()

In [85]:
l=[]
for d in date:
    df = pd.read_csv('D:\\datawhale\\joyful-pandas\\data\\us_report\\' + d + '.csv', index_col='Province_State')
    data=df.loc['New York',['Confirmed','Deaths','Recovered','Active']]
    l.append(data.to_frame().T)

res=pd.concat(l)#将一个个dataframe接到一起
res.index=date
res.head(3)

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887,179648
04-13-2020,195749,10058,23887,185691
04-14-2020,203020,10842,23887,192178


### EX2:实现join函数
请实现带有 how 参数的 join 函数
- 假设连接的两表无公共列
- 调用方式为 join(df1, df2, how="left")
- 给出测试样例

In [90]:
df1 = pd.DataFrame({'col1':list('01234')}, index=list('AABCD'))
df2 = pd.DataFrame({'col2':list('opqrst')}, index=list('ABBCEE'))

In [105]:
#how参数有四种，这个how得做一个判断
def join(df1,df2,how='left'):
    res_col = df1.columns.tolist() +  df2.columns.tolist()#两个字符串相加组成一个list
    dup = df1.index.unique().intersection(df2.index.unique())#两个dataframe的独一无二的索引的交集ABC
    res_df = pd.DataFrame(columns = res_col)#构建一个列名为df1，df2所有列名的一个dataframe
    for label in dup:#处理交集索引
        cartesian = [list(i)+list(j) for i in df1.loc[label].values for j in df2.loc[label].values]
        #print(cartesian)[['0', 'o'], ['1', 'o']]
                        #[['2', 'p'], ['2', 'q']]
                        #[['3', 'r']]
        #交集索引写到输出结果里面是AABBC
        dup_df = pd.DataFrame(cartesian, index = [label]*len(cartesian), columns = res_col)
        res_df = pd.concat([res_df,dup_df])
    if how in ['left', 'outer']:#分情况处理how，为lefter/outer的时候
        for label in df1.index.unique().difference(dup):#差集，在df1中，不在dup里面
            if isinstance(df1.loc[label], pd.DataFrame):
                cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].values]
            else: cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].to_frame().values]
            #cat定义其他行放什么放进去新建的数组
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    if how in ['right', 'outer']:
        for label in df2.index.unique().difference(dup):
            if isinstance(df2.loc[label], pd.DataFrame):
                cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].values]
            else: cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].to_frame().values]
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    return res_df

join(df1, df2, how='outer')

,col1,col2
A,0,o
A,1,o
B,2,p
B,2,q
C,3,r
D,4,NaN
E,NaN,s
E,NaN,t
